In [0]:
!pip install tqdm

In [0]:
%restart_python

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('/Volumes/estudo/default/ec/df_t.csv')
df.head()

In [0]:
tabela_modelo_previsao_rota = "estudo.default.tprev_rota_silver"

In [0]:
df['purchase_datetime'] = pd.to_datetime(df['date_purchase'] + ' ' + df['time_purchase'])

df.drop(['date_purchase', 'time_purchase'], axis=1, inplace=True)

df.sort_values(by='purchase_datetime', inplace=True)

In [0]:
import numpy as np

df.replace('0', np.nan, inplace=True)

print(df.isnull().sum())

In [0]:
df['trecho'] = df['place_origin_departure'] + ' -> ' + df['place_destination_departure']

print(df[['place_origin_departure', 'place_destination_departure', 'trecho']].head())

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

top_10_trechos = df['trecho'].value_counts().head(10)

plt.figure(figsize=(8, 4))
sns.barplot(x=top_10_trechos.values, y=top_10_trechos.index, palette='viridis')
plt.title('Top 10 Trechos Mais Populares')
plt.xlabel('Número de Vendas')
plt.ylabel('Trecho (Origem -> Destino)')
plt.show()

In [0]:
top_10_clientes = df['fk_contact'].value_counts().head(10)
print("Top 10 Clientes Mais Frequentes:\n", top_10_clientes)

In [0]:
compras_por_usuario = df.groupby('fk_contact')['trecho'].count()

plt.figure(figsize=(10, 6))
sns.histplot(compras_por_usuario, bins=30, kde=True)
plt.title('Distribuição de Compras por Usuário')
plt.xlabel('Número de Compras')
plt.ylabel('Número de Usuários')
plt.xlim(0, 20)
plt.show()

In [0]:
# Definir uma data de corte 80% do tempo para treino, 20% para teste
data_corte = df['purchase_datetime'].quantile(0.8, interpolation='nearest')

df_treino = df[df['purchase_datetime'] < data_corte]
df_teste = df[df['purchase_datetime'] >= data_corte]

print(f"Dados de treino: {df_treino.shape[0]} registros, terminando em {df_treino['purchase_datetime'].max()}")
print(f"Dados de teste: {df_teste.shape[0]} registros, começando em {df_teste['purchase_datetime'].min()}")

In [0]:
recomendacoes_por_cliente_fallback = df_treino.groupby('fk_contact')['trecho'].apply(
    lambda x: x.value_counts().head(5).index.tolist()
).to_dict()

recomendacoes_por_origem = df_treino.groupby('place_origin_departure')['trecho'].apply(
    lambda x: x.value_counts().head(5).index.tolist()
).to_dict()

top_5_trechos_gerais = df_treino['trecho'].value_counts().head(5).index.tolist()


# --- Lógica Principal: Baseada no Próximo Trecho Mais Provável ---
df_treino_sorted = df_treino.sort_values(by=['fk_contact', 'purchase_datetime'])
df_treino_sorted['next_trecho'] = df_treino_sorted.groupby('fk_contact')['trecho'].shift(-1)
transitions = df_treino_sorted.dropna(subset=['next_trecho'])
next_trecho_populares = transitions.groupby('trecho')['next_trecho'].apply(
    lambda x: x.value_counts().head(5).index.tolist()
).to_dict()
last_trip_per_user = df_treino_sorted.drop_duplicates(subset='fk_contact', keep='last').set_index('fk_contact')['trecho'].to_dict()


def gerar_recomendacoes_v3(fk_contact, origin_place=None):
    """
    Gera recomendações com lógica sequencial e fallback contextual por origem para clientes novos.
    """
    if fk_contact in last_trip_per_user:
        last_trip = last_trip_per_user[fk_contact]
        if last_trip in next_trecho_populares:
            return next_trecho_populares[last_trip]

    if fk_contact in recomendacoes_por_cliente_fallback:
        return recomendacoes_por_cliente_fallback[fk_contact]

    if origin_place and origin_place in recomendacoes_por_origem:
        return recomendacoes_por_origem[origin_place]

    return top_5_trechos_gerais

cliente_existente_exemplo = df_treino['fk_contact'].iloc[0]
cliente_novo_exemplo = 'cliente_novo_teste_12345'
origem_exemplo = df_teste['place_origin_departure'].iloc[0]

print(f"Recomendações para o cliente existente ({cliente_existente_exemplo[:10]}...):")
print(gerar_recomendacoes_v3(cliente_existente_exemplo))
print("\n" + "="*50 + "\n")
print(f"Recomendações para o cliente novo ({cliente_novo_exemplo}) partindo de ({origem_exemplo[:10]}...):")
print(gerar_recomendacoes_v3(cliente_novo_exemplo, origin_place=origem_exemplo))


In [0]:
def calcular_hit_rate(df_teste, func_recomendacao, top_k=5):
    """
    Calcula o Hit Rate @K para um conjunto de dados de teste usando uma função de recomendação.
    """
    import tqdm
    
    total_compras_teste = len(df_teste)
    total_hits = 0
    
    for _, compra in tqdm.tqdm(df_teste.iterrows(), total=total_compras_teste, desc="Avaliando o modelo"):
        fk_contact = compra['fk_contact']
        trecho_real = compra['trecho']
        origem_real = compra['place_origin_departure']
        
        recomendacoes = func_recomendacao(fk_contact, origin_place=origem_real)
        
        if trecho_real in recomendacoes:
            total_hits += 1
    
    hit_rate = (total_hits / total_compras_teste) if total_compras_teste > 0 else 0
    
    return {
        'hit_rate': hit_rate,
        'total_hits': total_hits,
        'total_registros': total_compras_teste
    }


In [0]:
resultado_avaliacao = calcular_hit_rate(df_teste, gerar_recomendacoes_v3)

print(f"\nCálculo Finalizado:")
print(f"Total de Compras no Conjunto de Teste: {resultado_avaliacao['total_registros']}")
print(f"Total de Hits @5: {resultado_avaliacao['total_hits']}")

hit_rate = resultado_avaliacao['hit_rate']


In [0]:
# Apresentação do resultado
print(f"O Hit Rate @5 do modelo baseline é: {hit_rate:.2%}")
